In [ ]:
%pip install qiskit
%pip install pylatexenc
%pip install qiskit-aer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pylatexenc: filename=pylatexenc-2.10-py3-none-any.whl size=136817 sha256=1d4fc617769d0dbdd333c7987669b82d5d3e30ac94da1885a986b4ee5c64b426
  Stored in directory: /root/.cache/pip/wheels/06/3e/78/fa1588c1ae991bbfd814af2bcac6cef7a178beee1939180d46
Successfully built pylatexenc
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 98.4 MB/s eta 0:00:00


In [ ]:
# ============================================================
# ONE-CELL: Correct CHSH re-simulation (No Eve vs Eve) for N list
# - Standard CHSH: S = |E(A,B)+E(A,b)+E(a,B)-E(a,b)|
# - CHSH-optimal angles: A=Z(0), a=X(pi/4), B=W(pi/8), b=V(-pi/8)
# - Prints both S and S/2 tables (mean ± std over many runs)
# ============================================================

import math, random
import numpy as np
import pandas as pd

# Angles in X–Z plane (measure by applying RY(-2*theta) then Z-measure)
theta = {
    "Z": 0.0,
    "X": math.pi/4,
    "W": math.pi/8,
    "V": -math.pi/8,
}

def basis_states(theta_):
    c, s = math.cos(theta_), math.sin(theta_)
    ket0 = np.array([c, s], dtype=complex)
    ket1 = np.array([-s, c], dtype=complex)
    return ket0, ket1

def sample_no_eve_phi_plus(thetaA, thetaB, rng):
    # |Phi+> = (|00>+|11>)/sqrt(2)
    phi = np.array([1,0,0,1], dtype=complex)/math.sqrt(2)
    A0,A1 = basis_states(thetaA)
    B0,B1 = basis_states(thetaB)

    probs = []
    outcomes = []
    for a,Ak in [(0,A0),(1,A1)]:
        for b,Bk in [(0,B0),(1,B1)]:
            ket = np.kron(Ak, Bk)
            amp = np.vdot(ket, phi)
            p = float(abs(amp)**2)
            probs.append(p)
            outcomes.append((a,b))
    probs = np.array(probs, dtype=float)
    probs = probs / probs.sum()

    r = rng.random()
    cum = 0.0
    for (a,b),p in zip(outcomes, probs):
        cum += p
        if r <= cum:
            return a,b
    return outcomes[-1]

def measure_1q(state, thetaA, rng):
    ket0, ket1 = basis_states(thetaA)
    p0 = float(abs(np.vdot(ket0, state))**2)
    if rng.random() < p0:
        return 0
    return 1

def eve_intercept_resend(thetaE, rng):
    # Reduced state is maximally mixed -> Eve gets 0/1 uniformly
    e = 0 if rng.random() < 0.5 else 1
    ket0, ket1 = basis_states(thetaE)
    st = ket0 if e == 0 else ket1
    # After Eve measurement, Alice collapses to same eigenstate for |Phi+>
    return e, st, st

def chsh_one_run(N, seed=0, eve=False, eve_basis_set=("Z","X")):
    rng = random.Random(seed)

    # CHSH settings
    A, a = "Z", "X"
    B, b = "W", "V"
    settings = [(A,B), (A,b), (a,B), (a,b)]

    # allocate shots across 4 settings
    base = N // 4
    rem  = N % 4
    shots = [base + (1 if i < rem else 0) for i in range(4)]

    Es = []
    for (Abasis, Bbasis), sh in zip(settings, shots):
        vals = []
        for _ in range(sh):
            if not eve:
                aa, bb = sample_no_eve_phi_plus(theta[Abasis], theta[Bbasis], rng)
            else:
                e_basis = rng.choice(list(eve_basis_set))
                e, alice_state, bob_state = eve_intercept_resend(theta[e_basis], rng)
                aa = measure_1q(alice_state, theta[Abasis], rng)
                bb = measure_1q(bob_state,   theta[Bbasis], rng)

            A_pm = +1 if aa == 0 else -1
            B_pm = +1 if bb == 0 else -1
            vals.append(A_pm * B_pm)

        Es.append(sum(vals)/len(vals))

    E_AB, E_Ab, E_aB, E_ab = Es

    # ✅ Correct CHSH:
    S = abs(E_AB + E_Ab + E_aB - E_ab)
    return S

def aggregate(N_list, repeats=500, eve=False, seed_base=1234):
    rows=[]
    for N in N_list:
        Ss=[]
        for r in range(repeats):
            Ss.append(chsh_one_run(N, seed=seed_base + N*10000 + r, eve=eve))
        rows.append({
            "N": N,
            "S_mean": float(np.mean(Ss)),
            "S_std":  float(np.std(Ss, ddof=1)),
            "S/2_mean": float(np.mean(Ss)/2),
            "S/2_std":  float(np.std(Ss, ddof=1)/2),
        })
    return pd.DataFrame(rows)

N_LIST = [50, 150, 350, 500]

df_no  = aggregate(N_LIST, repeats=500, eve=False)
df_eve = aggregate(N_LIST, repeats=500, eve=True)

print("\nNO EVE (standard S):")
print(df_no[["N","S_mean","S_std"]])

print("\nWITH EVE (standard S):")
print(df_eve[["N","S_mean","S_std"]])

print("\nNO EVE (normalized S/2):")
print(df_no[["N","S/2_mean","S/2_std"]])

print("\nWITH EVE (normalized S/2):")
print(df_eve[["N","S/2_mean","S/2_std"]])



NO EVE (standard S):
     N    S_mean     S_std
0   50  2.818718  0.407472
1  150  2.826597  0.230740
2  350  2.832536  0.141473
3  500  2.822784  0.125952

WITH EVE (standard S):
     N    S_mean     S_std
0   50  1.420513  0.523999
1  150  1.399223  0.291666
2  350  1.412838  0.211113
3  500  1.420512  0.167113

NO EVE (normalized S/2):
     N  S/2_mean   S/2_std
0   50  1.409359  0.203736
1  150  1.413299  0.115370
2  350  1.416268  0.070737
3  500  1.411392  0.062976

WITH EVE (normalized S/2):
     N  S/2_mean   S/2_std
0   50  0.710256  0.262000
1  150  0.699612  0.145833
2  350  0.706419  0.105556
3  500  0.710256  0.083556
